# Make sure your runtime has a gpu!

In [1]:
# Clone transformer explainbility repo, install other dependencies
!git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os;
os.chdir(f'./Transformer-Explainability')
  
%pip install -r requirements.txt
%pip install captum
%pip install gensim

from utils.kmers import kMerDF, kMer
import utils.kmers as utils
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
from Bio import SeqIO
import torch

from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import AutoTokenizer, TextClassificationPipeline

Cloning into 'Transformer-Explainability'...
remote: Enumerating objects: 377, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 377 (delta 127), reused 74 (delta 74), pack-reused 225
Receiving objects: 100% (377/377), 3.83 MiB | 3.83 MiB/s, done.
Resolving deltas: 100% (190/190), done.
on a case-insensitive filesystem) and only one from the same
colliding group is in the working tree:

  'data/Imagenet.py'
  'data/imagenet.py'
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'utils.kmers'

In [12]:
#download pre-trained model off of huggingface

model = BertForSequenceClassification.from_pretrained('AidenH20/DNABERT-500down', id2label={0: 'negative', 1: 'positive'}).to('cuda')
model.eval()

tokenizer = AutoTokenizer.from_pretrained('AidenH20/DNABERT-500down')

Downloading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 485kB/s]
Downloading: 100%|██████████| 357M/357M [00:32<00:00, 10.9MB/s] 


AssertionError: Torch not compiled with CUDA enabled

In [14]:
%%time
explanations = Generator(model)

classifications = ['unstable', 'stable']

def gen_explanation(text_batch, true_class):
  """This function is adapted from Chefer et al's. BERT-explainibility 
  notebook. Link to their github repo is here: 
  https://github.com/hila-chefer/Transformer-Explainability
  """
  # encode a sentence
  encoding = tokenizer(text_batch, return_tensors='pt')
  input_ids = encoding['input_ids'].to("cuda")
  attention_mask = encoding['attention_mask'].to("cuda")
  
  # generate an explanation for the input
  expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]

  # get the model classification
  output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
  classification = output.argmax(dim=-1).item()

  tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
  return [(tokens[i], expl[i].item()) for i in range(len(tokens))]

%cd /examples
result = pd.DataFrame(pd.read_csv('XLTestingData.csv'))

seqs = result[['b3', 'label']]

%cd /512ds
with open('512expl.csv', 'a') as f:
  f.write('scores,' + 'label' + '\n')
i = 0
while i < len(seqs):
  with open('512expl.csv', 'a') as f:
    expl = gen_explanation(text_batch = seqs['b3'][i], true_class = seqs['label'][i])
    expl = [item[1] for item in expl[1:-1]]
    expl = ' '.join(map(str, expl))
    string = '{0},{1}\n'.format(str(expl), str(seqs['label'][i]))
    f.write(string)
    i += 1

NameError: name 'model' is not defined

In [18]:
# temporary
%cd ../examples/512ds

/Users/aidenh20/Polylong/examples/512ds


In [19]:
#Load attention scores, convert to numpy array
expl_scores = pd.read_csv('512expl.csv')['scores']
expl_scores = [str(i).split(' ') for i in expl_scores]
expl_scores = np.array(expl_scores).astype(float)

validation_labels = np.array(pd.read_csv('512expl.csv')['label'])

#load DNA sequences and their labels, convert to numpy arrays.
dnadf = pd.read_csv('../XLTestingData.csv')['b3']
dnadf = np.array([str(i).split(' ') for i in dnadf])
labels = pd.read_csv('../XLTestingData.csv')['label'].to_numpy()


#make sure all the sequences in both dataframes are sorted in the same order
assert validation_labels.all() == labels.all()

In [20]:
#create the kMerDF object
X = kMerDF(expl_scores, dnadf, labels)

#get the kMer object at row 0, column 506
print('kMerDF[0][506]: ' + X[0][506].seq)

#showing this is the same kmer as the first in the dnadf
print('dnadf[0][506]: ' + dnadf[0][506])

kMerDF[0][506]: CAGGCG
dnadf[0][506]: CAGGCG


In [ ]:
def get_kmers_matching_region(kmer, kmerdf, start, end):
    """Returns a list of kMers that match the region of the kMer passed in.
    """
    matching_kmers = []
    for i in range(start, end):
        if kmerdf[i].seq == kmer.seq:
            matching_kmers.append(kmerdf[i])
    return matching_kmers


In [21]:
with open('contig_kmers.fa', 'a') as f:
    for l in X.get_contiguous_kmers(zscore = 1): #get contiguous regions with z-score >= 1.
        if l != [] and l[0].pos <= 100: #TODO: weird behavior of get_contiguous_kmers with the empty lists
            # write a unique sequence header
            f.write('>ID' + str(l[0].ID) + 'label' + str(l[0].label) + ':' + str(l[0].pos) + '\n')
            r_seq = ''
            for kmer in l: #l is a list of kMer objects
                r_seq += kmer.seq + ' '
            f.write(utils.kmer2seq(r_seq.strip()) + '\n')

# TODO: check the other bins, so positions 100-200, 200-300, 300-400, 400-507. 

Here, I plug in the results from the previous cell into DREME, using positive sequences as a query and negative as the control.